Created 09/01/2026 by truji@mit.edu

A simple candidate filter for picking targets. 
Takes observatory code and list of possible targets from, in my case, the NEA Observation Planning Aid, and narrows them down based on observing window etc.
Uses JPL Horizons to do this.

In [1]:
from astroquery.jplhorizons import Horizons
from astropy import units as u
from astropy.coordinates import AltAz, EarthLocation, SkyCoord, get_body, get_sun
from astropy.time import Time
import re
from tqdm import tqdm

In [2]:
# make list of object names by copying from NEA Observation Planning Aid output

file_path = "possible_targets.txt"
with open(file_path, "r") as file:
    raw = file.read()

ids = [int(m.group(1)) for m in re.finditer(r'(?m)^\s*\((\d+)\)', raw)]

In [4]:
# set obs time
utc = Time("2026-01-15 02:00:00", scale='utc')

# get target altitudes
all = {}
for n, name in tqdm(enumerate(ids), total=len(ids)):
    obj = Horizons(name, location=954, epochs=[utc.tdb.jd])
    tab = obj.ephemerides()

    # Horizons package can't handle physical properties. Find rotper with regex on raw api out
    raw = str(obj.ephemerides_async()._content)
    match = re.search(r"ROTPER=\s*([0-9.]+)", raw)
    rotper = float(match.group(1)) if match else 1e6

    # update dict with all needed info
    all[name] = (float(tab['EL'][0]), float(tab['hour_angle'][0]), float(tab['V'][0]), rotper)

100%|██████████| 310/310 [00:22<00:00, 13.94it/s]


In [9]:
# review top candidates. best is determined by alt / mag
cutoff = sorted([value[0] / value[2] for value in all.values()])[-20]

best_targets = {} # save as csv using vscode datawrangler
for name, value in all.items():
    if value[0] / value[2] >= cutoff and value[3] == 1e6:
        print(f"{name} will be at alt {value[0]} and hour angle {value[1]} at {utc}. It is magnitude {value[2]} and has rotper {value[3]}")
        best_targets[name] = value

836372 will be at alt 70.751112 and hour angle 0.852049469 at 2026-01-15 02:00:00.000. It is magnitude 16.272 and has rotper 1000000.0
137802 will be at alt 62.600285 and hour angle 1.617007632 at 2026-01-15 02:00:00.000. It is magnitude 16.914 and has rotper 1000000.0
533722 will be at alt 82.868592 and hour angle -0.489299932 at 2026-01-15 02:00:00.000. It is magnitude 17.108 and has rotper 1000000.0
141765 will be at alt 76.824764 and hour angle 0.059735352 at 2026-01-15 02:00:00.000. It is magnitude 18.082 and has rotper 1000000.0
528756 will be at alt 82.609757 and hour angle -0.042081942 at 2026-01-15 02:00:00.000. It is magnitude 18.466 and has rotper 1000000.0
213053 will be at alt 79.873278 and hour angle -0.366631289 at 2026-01-15 02:00:00.000. It is magnitude 18.677 and has rotper 1000000.0
614313 will be at alt 80.453311 and hour angle -0.720315022 at 2026-01-15 02:00:00.000. It is magnitude 18.989 and has rotper 1000000.0
204232 will be at alt 80.25681 and hour angle -0.64